<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 11:59:59
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -89.63 K (-0.65%)
Current PnL: -37.10 L (-23.19%)
CY Booked + Current PnL: -21.42 L (-13.39%)
-------------------
Total profit:  1.44 L
Total loss:  -38.54 L
-------------------
Total Booked + Current PnL: 5.56 L (4.18%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.42%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (75.52%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.50,5.00,12.06,17.67,19054.0,7530.0,157990.0,478.51,26.26,53.0,L-LC,10.99,185.0,0.40,1.17,26.83,XY25,ATH,MINING
50,MASFIN,-2.86,0.80,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE
77,TTKPRESTIG,-0.62,-31.89,46.96,0.09,32232.0,-32140.0,68637.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-1.54,-4.50,29.92,24.08,36276.0,-5707.0,121243.0,1486.00,-5.20,50.0,X-SC,7.97,87.0,-0.16,0.90,11.05,XY24,NTT,HEALTHCARE
37,ICICIGI,-2.02,1.02,18.13,19.34,36650.0,2044.0,202153.0,2252.93,-14.93,49.0,X-MC,1.09,66.0,0.06,1.50,17.12,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,7.12,18.49,75.47,107.91,132026.0,27294.0,174938.0,1641.55,6.54,77.0,M-SC,7.03,220.0,0.21,1.30,31.61,OX40N,ATH,CABLES
24,DIXON,4.04,-27.03,79.87,31.26,134501.0,-62368.0,168400.0,18931.72,-67.41,40.0,X-MC,24.11,54.0,-0.46,1.25,4.04,X40N,ATH,IT
49,LAOPALA,2.26,-37.50,122.54,39.09,100409.0,-49165.0,81940.0,464.00,79.36,59.0,H-SC,13.39,154.0,-0.49,0.61,9.58,AR,NTT,CERAMICS
69,SURYODAY,1.73,-27.08,68.74,23.03,89754.0,-48501.0,130570.0,216.00,42.54,48.0,H-SC,5.63,166.0,-0.54,0.97,29.55,XR,NTT,BANKS
82,VOLTAS,1.46,22.59,18.44,45.20,43343.0,43308.0,235050.0,1856.00,40.38,66.0,X-MC,9.10,79.0,1.00,1.74,30.92,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,-3.34,-10.26,38.71,24.48,87491.0,-25839.0,226017.0,950.00,-13.91,36.0,X-SC,5.64,89.0,-0.30,1.68,11.06,X40N,NTT,MISC
11,BANDHANBNK,-3.18,-13.33,122.00,92.42,294558.0,-37118.0,241441.0,400.00,106.90,68.0,H-SC,8.06,174.0,-0.13,1.79,33.00,XY24,NTT,BANKS
55,RAJESHEXPO,-3.07,-71.73,253.95,0.07,98489.0,-98394.0,38783.0,518.00,1321.72,31.0,L-SC,9.90,268.0,-1.00,0.29,0.00,OX40N,NTT,JEWELLERY
50,MASFIN,-2.86,0.80,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE
38,ICICIPRULI,-2.75,8.55,20.43,30.73,43691.0,16851.0,213856.0,790.00,-20.57,51.0,X-MC,4.15,78.0,0.39,1.59,22.41,X40,ATH,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-2.86,0.8,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,7.12,18.49,75.47,107.91,132026.0,27294.0,174938.0,1641.55,6.54,77.0,M-SC,7.03,220.0,0.21,1.30,31.61,OX40N,ATH,CABLES
66,SIS,-0.45,-30.97,75.68,21.26,58001.0,-34392.0,76640.0,528.00,1798.96,38.0,H-SC,5.60,168.0,-0.59,0.57,5.66,OX40N,NTT,MISC
77,TTKPRESTIG,-0.62,-31.89,46.96,0.09,32232.0,-32140.0,68637.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
41,INDIGOPNTS,-0.39,-32.96,49.18,0.02,57556.0,-57528.0,117031.0,1408.00,78.57,27.0,M-SC,12.16,222.0,-1.00,0.87,1.80,OX40N,NTT,PAINTS
48,KANSAINER,-0.39,-33.38,70.32,13.47,126336.0,-90009.0,179658.0,340.00,-42.69,32.0,H-SC,13.08,162.0,-0.71,1.33,0.99,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.46,22.59,18.44,45.2,43343.0,43308.0,235050.0,1856.0,40.38,66.0,X-MC,9.1,79.0,1.0,1.74,30.92,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.50,5.00,12.06,17.67,19054.0,7530.0,157990.0,478.51,26.26,53.0,L-LC,10.99,185.0,0.40,1.17,26.83,XY25,ATH,MINING
27,FINCABLES,7.12,18.49,75.47,107.91,132026.0,27294.0,174938.0,1641.55,6.54,77.0,M-SC,7.03,220.0,0.21,1.30,31.61,OX40N,ATH,CABLES
50,MASFIN,-2.86,0.80,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE
1,ABB,-0.65,11.79,30.22,45.57,88383.0,30845.0,292464.0,7934.00,-28.16,66.0,H-MC,8.81,125.0,0.35,2.17,29.87,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.65,11.79,30.22,45.57,88383.0,30845.0,292464.0,7934.00,-28.16,66.0,H-MC,8.81,125.0,0.35,2.17,29.87,AR,NTT,ELECTRICAL
27,FINCABLES,7.12,18.49,75.47,107.91,132026.0,27294.0,174938.0,1641.55,6.54,77.0,M-SC,7.03,220.0,0.21,1.30,31.61,OX40N,ATH,CABLES
50,MASFIN,-2.86,0.80,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE
85,ZYDUSLIFE,-1.68,-2.69,39.38,35.63,80307.0,-5646.0,203928.0,1286.17,-15.32,55.0,H-MC,4.89,120.0,-0.07,1.51,13.53,AR,ATH,PHARMA
40,INDIAMART,-1.23,-7.44,123.33,106.72,140796.0,-9174.0,114162.0,4810.62,-58.20,41.0,H-SC,8.08,140.0,-0.07,0.85,14.20,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,0.05,-38.58,155.94,57.19,312351.0,-125825.0,200302.0,485.00,-64.88,19.0,X-SC,18.82,81.0,-0.40,1.48,0.05,XY24,NTT,FMCG
34,HCLTECH,-0.34,-10.86,38.93,23.83,83949.0,-26284.0,215640.0,1908.19,-6.53,26.0,X-LC,5.40,8.0,-0.31,1.60,3.46,X40,ATH,IT
43,INFY,1.30,-11.11,51.02,34.24,141904.0,-34753.0,278135.0,1972.00,-29.97,26.0,X-LC,8.69,3.0,-0.24,2.06,2.38,X40,NTT,IT
12,BATAINDIA,-0.59,-49.04,165.32,35.20,108400.0,-63100.0,65570.0,2096.00,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
53,PGHH,-0.17,-14.33,54.09,32.01,98946.0,-30592.0,182928.0,17616.87,-42.03,30.0,X-MC,9.59,57.0,-0.31,1.36,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-2.02,1.02,18.13,19.34,36650.0,2044.0,202153.0,2252.93,-14.93,49.0,X-MC,1.09,66.0,0.06,1.50,17.12,X40,ATH,INSURANCE
78,UNITDSPR,-1.68,2.02,18.38,20.77,43905.0,4730.0,238874.0,1644.00,-9.55,50.0,X-MC,1.12,64.0,0.11,1.77,9.57,X40N,NTT,BREWERIES
22,DABUR,-1.39,1.11,42.13,43.70,105670.0,2750.0,250818.0,735.00,-8.20,53.0,X-MC,1.46,70.0,0.03,1.86,16.33,XY24,BTT,FMCG
59,RELIANCE,-0.06,-9.75,38.84,25.30,70441.0,-19602.0,181361.0,1952.00,-11.96,42.0,X-LC,2.29,21.0,-0.28,1.34,21.48,XY25,BTT,REFINERIES
5,ANGELONE,-2.35,-90.14,1160.07,24.20,349042.0,-275162.0,30088.0,3033.00,-85.87,38.0,X-SC,2.51,97.0,-0.79,0.22,26.01,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.61,-12.12,34.36,18.07,54004.0,-21685.0,157170.0,35195.0,-25.27,36.0,X-MC,11.51,58.0,-0.40,1.17,0.0,X40,ATH,PHARMA
3,ACC,-1.18,-33.01,145.04,64.15,231194.0,-78551.0,159400.0,3906.0,-49.40,33.0,X-SC,9.50,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
12,BATAINDIA,-0.59,-49.04,165.32,35.20,108400.0,-63100.0,65570.0,2096.0,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,-0.75,-25.55,108.84,55.47,197131.0,-62166.0,181120.0,181.5,-36.04,32.0,X-MC,17.21,60.0,-0.32,1.34,0.0,X40N,ATH,FINANCE
58,RELAXO,-0.66,-54.44,234.28,52.31,154948.0,-79022.0,66138.0,1176.0,-51.51,37.0,X-SC,14.78,92.0,-0.51,0.49,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-2.35,-90.14,1160.07,24.20,349042.0,-275162.0,30088.0,3033.0,-85.87,38.0,X-SC,2.51,97.0,-0.79,0.22,26.01,X40N,NTT,FINANCE
54,QUESS,-0.81,-35.10,120.10,42.84,50668.0,-22818.0,42188.0,424.0,-57.58,35.0,X-SC,17.12,80.0,-0.45,0.31,1.48,XY24,NTT,MISC
58,RELAXO,-0.66,-54.44,234.28,52.31,154948.0,-79022.0,66138.0,1176.0,-51.51,37.0,X-SC,14.78,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.59,-49.04,165.32,35.20,108400.0,-63100.0,65570.0,2096.0,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-1.54,-4.50,29.92,24.08,36276.0,-5707.0,121243.0,1486.0,-5.20,50.0,X-SC,7.97,87.0,-0.16,0.90,11.05,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.77,-16.80,43.11,19.06,122002.0,-57164.0,283002.0,452.00,-54.31,41.0,X-LC,17.18,1.0,-0.47,2.10,2.47,X40,NTT,FMCG
73,TCS,-0.06,-24.16,62.95,23.58,206541.0,-104525.0,328104.0,4311.58,-34.12,31.0,X-LC,9.13,2.0,-0.51,2.43,2.81,X40,ATH,IT
43,INFY,1.30,-11.11,51.02,34.24,141904.0,-34753.0,278135.0,1972.00,-29.97,26.0,X-LC,8.69,3.0,-0.24,2.06,2.38,X40,NTT,IT
75,TMPV,-0.91,-13.78,54.64,33.33,153490.0,-44888.0,280912.0,600.00,-81.44,63.0,X-LC,3.95,4.0,-0.29,2.08,14.84,XY24,NTT,AUTO
34,HCLTECH,-0.34,-10.86,38.93,23.83,83949.0,-26284.0,215640.0,1908.19,-6.53,26.0,X-LC,5.40,8.0,-0.31,1.60,3.46,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.58,-29.20,87.69,32.89,47937.0,-22540.0,54666.0,1800.00,-357.00,64.0,L-MC,12.11,258.0,-0.47,0.41,50.55,XR,NTT,BANKS
67,SONACOMS,-1.94,-7.49,50.35,39.09,47118.0,-7578.0,93581.0,804.02,-26.68,59.0,M-MC,10.83,193.0,-0.16,0.69,32.39,AR,ATH,AUTO
50,MASFIN,-2.86,0.80,20.61,21.57,20354.0,780.0,98760.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.12,XR,ATH,FINANCE
19,COALINDIA,-1.50,5.00,12.06,17.67,19054.0,7530.0,157990.0,478.51,26.26,53.0,L-LC,10.99,185.0,0.40,1.17,26.83,XY25,ATH,MINING
65,SIEMENS,-0.36,-10.67,40.50,25.51,67329.0,-19850.0,166245.0,4671.50,39.61,61.0,H-LC,3.47,51.0,-0.29,1.23,22.51,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-0.00,-52.10,204.15,45.70,172913.0,-92116.0,84699.0,26.40,-11.99,62.0,M-SC,18.18,204.0,-0.53,0.63,39.55,XY24,NTT,TRAVEL
27,FINCABLES,7.12,18.49,75.47,107.91,132026.0,27294.0,174938.0,1641.55,6.54,77.0,M-SC,7.03,220.0,0.21,1.30,31.61,OX40N,ATH,CABLES
1,ABB,-0.65,11.79,30.22,45.57,88383.0,30845.0,292464.0,7934.00,-28.16,66.0,H-MC,8.81,125.0,0.35,2.17,29.87,AR,NTT,ELECTRICAL
7,ATULAUTO,-0.87,-14.03,69.32,45.57,117479.0,-27659.0,169473.0,844.00,3672.97,59.0,M-SC,6.69,248.0,-0.24,1.26,28.25,XY24,NTT,AUTO
11,BANDHANBNK,-3.18,-13.33,122.00,92.42,294558.0,-37118.0,241441.0,400.00,106.90,68.0,H-SC,8.06,174.0,-0.13,1.79,33.00,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.80
1,25,45.82
2,50,77.83


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.00
MC    30.80
LC    25.23
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.48
X40      24.22
X40N     12.97
AR        9.33
XY25      9.10
XR        8.67
OX40N     7.40
SR        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.18
H-SC    22.68
X-LC    21.79
M-SC    11.06
X-SC     9.54
H-MC     5.09
M-MC     2.12
H-LC     1.23
L-LC     1.17
M-LC     1.04
L-SC     0.72
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.97,-14.54,53.28
IT,11.61,-40.10,110.31
FINANCE,11.20,-38.13,92.49
MISC,6.94,-38.80,95.58
ELECTRICAL,6.50,-7.05,46.11
PAINTS,5.12,-33.71,53.57
INSURANCE,4.98,-1.18,35.81
PHARMA,4.21,-3.48,36.05
AUTO,4.03,-14.73,58.48


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3419693.0,21
X40,1455782.0,15
X40N,1372184.0,10
XR,1322978.0,12
AR,1313129.0,10
OX40N,795697.0,10
XY25,388418.0,6
SR,303746.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3736709.0,24
X-SC,1516654.0,10
M-SC,1427794.0,14
X-MC,1387497.0,15
X-LC,1332166.0,12
H-MC,391763.0,3
L-SC,193992.0,2
M-LC,151660.0,1
M-MC,99072.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1267454.0      6
           AR         954487.0      5
           XR         847359.0      7
M-SC       XY24       838225.0      6
X-LC       X40        711295.0      6
X-SC       X40N       701235.0      4
           XY24       630489.0      4
X-MC       X40        559557.0      7
           X40N       442283.0      4
H-SC       OX40N      363663.0      4
M-SC       OX40N      333545.0      5
H-SC       SR         303746.0      2
X-LC       XY24       264812.0      2
           X40N       228666.0      2
H-MC       XY24       223073.0      1
X-MC       XY24       195640.0      2
           XY25       190017.0      2
X-SC       X40        184930.0      2
M-SC       XR         180519.0      2
H-MC       AR         168690.0      2
M-LC       XR         151660.0      1
X-LC       XY25       127393.0      2
L-SC       OX40N       98489.0      1
           XR          95503.0      1
M-SC       AR          75505.0      1
H-LC       AR          67329.0      1
M-MC       XY25        51954.0      1
L-MC       XR          47937.0      1
M-MC       AR          47118.0      1
L-LC       XY25        19054.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
